## run this cell before starting spark cluster connection (corrects module error)

In [1]:
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### select the data

In [4]:
df = spark.read.json("/user/rucio01/traces/traces.2018-05*")

### Create temporary table view

In [5]:
df.createOrReplaceTempView("traces")

### Use PySpark to query download events and split dataset DID into new features

In [6]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

### Official and User Data

In [7]:
df_ps = spark.sql("SELECT *\
                   FROM traces \
                   WHERE eventType=='download' AND clientState != 'ALREADY_DONE' AND dataset != 'None' AND dataset != '' AND dataset != '[null]' AND account != 'pilot'")\

In [8]:
df_ps.describe().toPandas()

summary _corrupt_record   account appid catStart     clientState  \
0  count   0               5322957   0     0        5322957          
1  mean    None            None      None  None     None             
2  stddev  None            None      None  None     None             
3  min     None            aalfonsi  None  None     AttributeError   
4  max     None            zzheng    None  None     SystemError      

                                                                                           dataset  \
0  5322957                                                                                           
1  None                                                                                              
2  None                                                                                              
3  13917493.events                                                                                   
4  valid2.361023.Pythia8EvtGen_A14NNPDF23LO_jetjet_JZ3W.merge.NTUP_PHYSVAL.e5112_s3265_r9723_p3370   

  datasetName datasetScope dataset_scope  duid eventType        eventVersion  \
0  0           5322913      0             0     5322957   5322957              
1  None        None         None          None  None      None                 
2  None        None         None          None  None      None                 
3  None        cond09_mc    None          None  download  1.13.2               
4  None        valid2       None          None  download  webui_1.16.0.post1   

                                                                                                                                       filename  \
0  5316562                                                                                                                                        
1  None                                                                                                                                           
2  None                                                                                                                                           
3  0.dat                                                                                                                                          
4  user.zzheng.410389.MadGraphPythia8EvtGen.DAOD_HIGG8D1.e6155_e5984_s3126_r9781_r9778_p3371.gn2_mc16c3L.2018-04-30.log.13918804.000005.log.tgz   

               filesize  guid                  hostname    ip localSite  \
0  5322913               0     5322938                   0     0          
1  2.813515787877685E8   None  None                      None  None       
2  1.0089260593584172E9  None  None                      None  None       
3  10000                 None  ReCaS-ui-01.cs.infn.it    None  None       
4  9999999               None  xenia.nevis.columbia.edu  None  None       

                                                                                           name  \
0  6370                                                                                           
1  None                                                                                           
2  None                                                                                           
3  NTUP_PILEUP.11325280._000001.pool.root.1                                                       
4  user.wichan.410470.PhPy8EG_A14_ttbar_hdamp258p75_nonallhad.14031080._000050.hist-output.root   

     pq protocol relativeStart                remoteSite                  rse  \
0  0     5322913  0             5322937                   38                    
1  None  None     None          None                      None                  
2  None  None     None          None                      None                  
3  None  davs     None          AGLT2_CALIBDISK           CERN-PROD_DATADISK    
4  None  srm      None          ZA-WITS-CORE_SCRATCHDISK  PRAGUELCG2_DATADISK   

       scope stateReason suspicious taskid timeEnd timeStart  \
0  5322

### drop columns that do not contain any values (or very little values, like 'name') based on previous describe()

In [9]:
df_ps = df_ps.drop('appid', 'catStart', 'datasetName', 'dataset_scope', 'duid', 
                   'guid', 'ip', 'localSite', 'pq', 'relativeStart', 'stateReason', 
                   'suspicious', 'taskid', 'timeEnd', 'timeStart', 'url', 'usr', 
                   'usrdn', 'validateStart', 'version', 'name')

## Drop empty Mai columns

In [10]:
df_ps = df_ps.drop('_corrupt_record', 'rse')

In [11]:
df_ps.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_ps.columns]).limit(10).toPandas()

account  clientState  dataset  datasetScope  eventType  eventVersion  \
0  0        0            0        44            0          0              

   filename  filesize  hostname  protocol  remoteSite  scope  traceId  \
0  6395      44        19        44        20          0      0         

   traceIp  traceTimeentry  traceTimeentryUnix  transferEnd  transferStart  \
0  0        0               0                   1398919      63              

   uuid  
0  0

## Drop transferend due to many missing values

In [12]:
df_ps = df_ps.drop('transferEnd')

In [13]:
df_ps = df_ps.dropna()

In [14]:
df_ps.createOrReplaceTempView("df_ps")

### Count dataset hits and join both tables

In [15]:
df_ps_counts = spark.sql("SELECT filename AS file_counts, count(*) as hits \
                               FROM traces \
                               WHERE eventType=='download' AND clientState != 'ALREADY_DONE' AND dataset != 'None' AND dataset != '' AND dataset != '[null]' AND account != 'pilot'\
                               GROUP BY  filename \
                               ORDER BY 2 DESC")

In [16]:
df_ps_counts.createOrReplaceTempView("counts")

In [17]:
df_ps_merged = spark.sql("SELECT * \
                               FROM df_ps \
                               INNER JOIN counts ON df_ps.filename = counts.file_counts")

In [18]:
df_ps_merged = df_ps_merged.drop('file_counts')

In [19]:
# refresh merged view so that all columns are in it
df_ps_merged.createOrReplaceTempView("merged")

In [20]:
df_ps_merged.count()

5316524

## cast filesize and run_number to double

In [21]:
df_ps_final = df_ps_merged.withColumn("filesize_double",df_ps_merged.filesize.cast(DoubleType()))

In [22]:
df_ps_final = df_ps_final.drop('filesize')

### Scale Numeric Features

In [23]:
df_ps_final.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_ps_final.columns]).limit(10).toPandas()

account  clientState  dataset  datasetScope  eventType  eventVersion  \
0  0        0            0        0             0          0              

   filename  hostname  protocol  remoteSite  scope  traceId  traceIp  \
0  0         0         0         0           0      0        0         

   traceTimeentry  traceTimeentryUnix  transferStart  uuid  hits  \
0  0               0                   0              0     0      

   filesize_double  
0  0

In [24]:
df_ps_final = df_ps_final.dropna()

In [25]:
df_ps_final.count()

5316524

In [26]:
df_ps_final.describe().toPandas()

summary   account     clientState  \
0  count   5316524   5316524          
1  mean    None      None             
2  stddev  None      None             
3  min     aalfonsi  AttributeError   
4  max     zzheng    SystemError      

                                                                                           dataset  \
0  5316524                                                                                           
1  None                                                                                              
2  None                                                                                              
3  13917493.events                                                                                   
4  valid2.361023.Pythia8EvtGen_A14NNPDF23LO_jetjet_JZ3W.merge.NTUP_PHYSVAL.e5112_s3265_r9723_p3370   

  datasetScope eventType eventVersion  \
0  5316524      5316524   5316524       
1  None         None      None          
2  None         None      None          
3  cond09_mc    download  1.13.2        
4  valid2       download  1.16.1        

                                                                                                                                       filename  \
0  5316524                                                                                                                                        
1  None                                                                                                                                           
2  None                                                                                                                                           
3  0.dat                                                                                                                                          
4  user.zzheng.410389.MadGraphPythia8EvtGen.DAOD_HIGG8D1.e6155_e5984_s3126_r9781_r9778_p3371.gn2_mc16c3L.2018-04-30.log.13918804.000005.log.tgz   

                   hostname protocol                remoteSite      scope  \
0  5316524                   5316524  5316524                   5316524     
1  None                      None     None                      None        
2  None                      None     None                      None        
3  ReCaS-ui-01.cs.infn.it    davs     AGLT2_CALIBDISK           cond09_mc   
4  xenia.nevis.columbia.edu  srm      ZA-WITS-CORE_SCRATCHDISK  valid2      

                            traceId                    traceIp  \
0  5316524                           5316524                     
1  Infinity                          None                        
2  NaN                               None                        
3  00000696ec754c998f70d84d737dfd18  2001:1458:201:e4::100:28c   
4  fffffe65b9c549e5ae746c15ec98d5bb  ::ffff:93.175.19.27         

               traceTimeentry    traceTimeentryUnix         transferStart  \
0  5316524                     5316524               5316524                
1  None                        1.5264044483840659E9  1.5264044110628998E9   
2  None                        735592.4639834111     735592.2918355962      
3  2018-04-30T22:00:01.400573  1.525125601400573E9   1.524860531819739E9    
4  2018-05-31T21:59:59.933752  1.527803999933752E9   1.527803999402263E9    

                               uuid                hits       filesize_double  
0  5316524                           5316524             5316524               
1  None                              6.517046664324284   2.8167336660135007E8  
2  None                              13.920048576048547  1.0094883391407579E9  
3  0000314863884f48a87c868ecdbe02f8  1                   2.0                   
4  fffee15801804450ad5c871d6ba40d6b  249                 4.7520668166E10

In [27]:
df_ps_final.createOrReplaceTempView("final")
df_hits = spark.sql("SELECT hits, count(*) \
                     FROM final \
                     GROUP BY hits")
df_hits_pd = df_hits.toPandas()

In [28]:
spark.sql("SELECT account, count(*) \
            FROM traces \
            WHERE eventType=='download' AND clientState != 'ALREADY_DONE' AND dataset != 'None' AND dataset != '' AND dataset != '[null]' AND account != 'pilot'\
            GROUP BY  account \
            ORDER BY 1 DESC").toPandas()

account  count(1)
0    zzheng     18889   
1    zuysal     11      
2    zmarshal   3       
3    zhidong    3       
4    zhelun     932     
5    zhangr     53      
6    zhangb     30      
7    zenon      35      
8    zchadi     4       
9    yyap       2825    
10   yunju      29      
11   yuj        65191   
12   yuhe       222     
13   yuchengh   4466    
14   yuchen     6       
15   ytakubo    1       
16   yshimoga   24      
17   ysano      112180  
18   ysaboo     14      
19   yqi        113     
20   yoyamagu   10      
21   yokazaki   1976    
22   ynoguchi   744     
23   ylo        266     
24   ykeisuke   7       
25   yilin      19      
26   yifang     110     
27   yhang      382     
28   yfukuhar   442     
29   yfhu       29      
30   yesw       20      
31   yesenia    18      
32   yelghaza   6       
33   ychow      363     
34   ychiu      1706    
35   yama       58826   
36   yabulait   300     
37   xuyang     2412    
38   xulou      160     
39   xsu        1265    
40   xingguo    300     
41   xiaohu     156     
42   xella      1       
43   xchen      18      
44   wyswys     3       
45   wsong      730     
46   woodsn     1526    
47   wleight    46      
48   wichan     21102   
49   wheidorn   4       
50   wguan      54      
51   wenhao     1358    
52   weitao     14164   
53   wdicleme   4218    
54   wchojnac   1056    
55   wbarbe     134     
56   watsona    896     
57   wangwe     20194   
58   walkerr    641     
59   walker     1       
60   waislam    108     
61   wachan     9       
62   vwong      677     
63   vvladimi   13      
64   vvecchio   891     
65   vtudorac   30      
66   vtskhaya   10      
67   vpetousi   1467    
68   vlang      1       
69   vkost      8       
70   vkitali    2925    
71   vippolit   5       
72   vincter    1036    
73   vdao       5       
74   vcavalie   27759   
75   vcairo     14773   
76   vbegy      51576   
77   vbailey    495     
78   vanjam     3279    
79   valentem   952     
80   vacavant   7       
81   uysasmaz   1       
82   umartens   48      
83   tzakarei   507     
84   tweston    102     
85   twegner    3       
86   tursom     1805    
87   turnerm    20      
88   tuna       5551    
89   ttork      95      
90   ttashiro   12      
91   tsfiligo   6162    
92   tsakai     51      
93   truggeri   7334    
94   trashid    59165   
95   tpowell    699     
96   tpoulsen   1       
97   tpelzer    113     
98   toyamaza   19893   
99   toshi      1       
100  tomoe      5       
101  tofitsch   5944    
102  todorova   422     
103  tnobe      274381  
104  tnitta     211217  
105  tneep      3904    
106  tmeideck   22      
107  tlyubush   4577    
108  tlari      65      
109  tkwan      60      
110  tkunigo    931     
111  tkodama    3672    
112  tkharlam   1563    
113  tkaji      50173   
114  thsteven   8       
115  thompson   115888  
116  tholmes    2       
117  tholm      214     
118  thmegy     1       
119  therwig    181     
120  tereshko   209     
121  temoskal   712     
122  tdiasdov   2       
123  tdado      2183    
124  tcuhadar   13      
125  tcontrer   1531    
126  tcharman   6428    
127  tburch     23      
128  tbeerman   1       
129  tavandaa   152     
130  tatsuya    161     
131  tapark     943     
132  tamartin   979     
133  tadej      149     
134  szerradi   528     
135  szambito   2306    
136  swahdan    534     
137  suyogs     1886    
138  sutt       71590   
139  susingh    8257    
140  stsuno     1       
141  sthais     3       
142  stephen    2433    
143  statsuyu   1       
144  sswift     1475    
145  ssuzuki    10347   
146  sspinali   3234    
147  ssolomon   837     
148  sshirabe   405     
149  sshaw      1       
150  sschmitt   164     
151  ssantpur   25      
152  ssahin     906     
153  srettie    9239    
154  sraspopo   61      
155  spieker    828     
156  sparajul   5548    
157  sosen      1829    
158  solovyev   7241    
159  sm

In [29]:
pd.set_option("display.max_rows",1900)
pd_sorted = df_hits_pd.sort_values('hits')
pd_sorted['cumsum'] = pd_sorted['count(1)'].cumsum(axis=0)
pd_sorted

hits  count(1)   cumsum
60   1     2271491   2271491
103  2     725116    2996607
86   3     241929    3238536
111  4     173968    3412504
59   5     141690    3554194
37   6     102918    3657112
13   7     202860    3859972
95   8     238403    4098375
43   9     101313    4199688
64   10    139160    4338848
99   11    111199    4450047
90   12    74952     4524999
114  13    47918     4572917
121  14    88396     4661313
125  15    40785     4702098
149  16    61184     4763282
51   17    20196     4783478
118  18    44982     4828460
4    19    169651    4998111
139  20    52940     5051051
123  21    70749     5121800
9    22    30030     5151830
136  23    54372     5206202
157  24    6504      5212706
31   25    6625      5219331
0    26    3406      5222737
44   27    3240      5225977
55   28    2352      5228329
1    29    2146      5230475
133  30    2640      5233115
27   31    2449      5235564
23   32    3008      5238572
56   33    1815      5240387
15   34    1904      5242291
101  35    1470      5243761
115  36    1800      5245561
88   37    1813      5247374
128  38    2470      5249844
29   39    1170      5251014
146  40    1240      5252254
54   41    1640      5253894
132  42    1512      5255406
24   43    1462      5256868
79   44    572       5257440
153  45    2160      5259600
138  46    552       5260152
154  47    470       5260622
74   48    816       5261438
100  49    588       5262026
19   50    350       5262376
46   51    1326      5263702
49   52    312       5264014
155  53    371       5264385
5    54    594       5264979
93   55    165       5265144
47   56    952       5266096
21   57    1083      5267179
41   58    464       5267643
124  59    472       5268115
143  60    3060      5271175
83   61    549       5271724
96   62    1054      5272778
45   63    1386      5274164
150  64    384       5274548
2    65    325       5274873
104  66    1980      5276853
73   67    804       5277657
38   68    136       5277793
120  69    414       5278207
140  70    210       5278417
34   71    142       5278559
39   72    720       5279279
134  73    730       5280009
94   74    222       5280231
116  75    450       5280681
108  76    228       5280909
14   77    231       5281140
117  78    468       5281608
52   79    632       5282240
102  80    80        5282320
126  81    324       5282644
129  82    410       5283054
89   83    415       5283469
25   84    84        5283553
69   85    765       5284318
40   87    87        5284405
57   88    88        5284493
66   89    89        5284582
135  90    270       5284852
151  91    364       5285216
145  93    186       5285402
18   94    188       5285590
62   96    288       5285878
131  97    388       5286266
141  99    594       5286860
76   100   100       5286960
87   101   303       5287263
109  102   1122      5288385
53   103   103       5288488
63   104   208       5288696
77   105   945       5289641
70   106   106       5289747
42   107   107       5289854
78   108   432       5290286
142  109   109       5290395
20   110   110       5290505
71   111   444       5290949
6    112   448       5291397
8    113   226       5291623
58   114   342       5291965
30   116   232       5292197
137  117   117       5292314
113  118   354       5292668
28   119   119       5292787
65   120   240       5293027
80   123   123       5293150
68   128   128       5293278
11   130   130       5293408
130  133   266       5293674
119  140   280       5293954
92   141   141       5294095
33   143   429       5294524
22   144   3888      5298412
152  152   152       5298564
61   153   153       5298717
7    155   620       5299337
85   156   2964      5302301
36   161   161       5302462
105  162   648       5303110
112  163   489       5303599
106  164   1804      5305403
75   168   672       5306075
156  169   338       5306413
98   171   171       5306584
50   174   174       5306758
110  175   175       5306933
107  176   528       5307461
72 

In [30]:
pd_sorted.plot(x='hits', y='cumsum')

/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/lib/python2.7/site-packages/matplotlib/font_manager.py:1316: UserWarning: findfont: Font family [u'sans-serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [31]:
df_ps_final = df_ps_final.drop("traceTimeentry")

In [32]:
categoricalFeatures = ['account', 'clientState', 'dataset', 'datasetScope', 'eventType', 'eventVersion', 'filename', 
                       'hostname', 'protocol', 'remoteSite', 'scope', 'traceId', 'traceIp', 'uuid']

encoded = df_ps_final

for elem in categoricalFeatures:
    encoded = encoded.withColumn(elem + "Hash", hash(encoded[elem]))

In [33]:
encoded.describe().toPandas()

summary   account     clientState  \
0  count   5316524   5316524          
1  mean    None      None             
2  stddev  None      None             
3  min     aalfonsi  AttributeError   
4  max     zzheng    SystemError      

                                                                                           dataset  \
0  5316524                                                                                           
1  None                                                                                              
2  None                                                                                              
3  13917493.events                                                                                   
4  valid2.361023.Pythia8EvtGen_A14NNPDF23LO_jetjet_JZ3W.merge.NTUP_PHYSVAL.e5112_s3265_r9723_p3370   

  datasetScope eventType eventVersion  \
0  5316524      5316524   5316524       
1  None         None      None          
2  None         None      None          
3  cond09_mc    download  1.13.2        
4  valid2       download  1.16.1        

                                                                                                                                       filename  \
0  5316524                                                                                                                                        
1  None                                                                                                                                           
2  None                                                                                                                                           
3  0.dat                                                                                                                                          
4  user.zzheng.410389.MadGraphPythia8EvtGen.DAOD_HIGG8D1.e6155_e5984_s3126_r9781_r9778_p3371.gn2_mc16c3L.2018-04-30.log.13918804.000005.log.tgz   

                   hostname protocol                remoteSite      scope  \
0  5316524                   5316524  5316524                   5316524     
1  None                      None     None                      None        
2  None                      None     None                      None        
3  ReCaS-ui-01.cs.infn.it    davs     AGLT2_CALIBDISK           cond09_mc   
4  xenia.nevis.columbia.edu  srm      ZA-WITS-CORE_SCRATCHDISK  valid2      

                            traceId                    traceIp  \
0  5316524                           5316524                     
1  Infinity                          None                        
2  NaN                               None                        
3  00000696ec754c998f70d84d737dfd18  2001:1458:201:e4::100:28c   
4  fffffe65b9c549e5ae746c15ec98d5bb  ::ffff:93.175.19.27         

     traceTimeentryUnix         transferStart  \
0  5316524               5316524                
1  1.5264044483840663E9  1.5264044110628994E9   
2  735592.4639834042     735592.2918356103      
3  1.525125601400573E9   1.524860531819739E9    
4  1.527803999933752E9   1.527803999402263E9    

                               uuid                hits       filesize_double  \
0  5316524                           5316524             5316524                
1  None                              6.517046664324284   2.8167336660135007E8   
2  None                              13.920048576048554  1.009488339140758E9    
3  0000314863884f48a87c868ecdbe02f8  1                   2.0                    
4  fffee15801804450ad5c871d6ba40d6b  249                 4.7520668166E10        

            accountHash       clientStateHash          datasetHash  \
0  5316524               5316524               5316524               
1  5.676034578304038E7   -8.556539364422513E8  8.513416665722266E7   
2  1.0782381847434075E9  3.393227998953657E8   1.264352081440324E9   
3  -2144719344           -1913538301           -2147477854           
4  2143592175            1269532

In [ ]:
encoded.write.format("json").save("/user/sprenner/training_all_mai.json")